# Complex Capacity Expansion

_**[Energy Systems Optimization](https://github.com/nspatank/energy_system_optimization)**_

_by Neha Patankar (last updated: November 16, 2023)_

This notebook, the final in our series, combines elements of several prior models, including [Basic Capacity Expansion](03-Basic-Capacity-Expansion.ipynb), [Economic Dispatch](04-Economic-Dispatch.ipynb), and a [Network Flow](06-Optimal-Power-Flow.ipynb) model, to demonstrate a more complex capacity expansion planning model that includes chronologically sequential hourly economic dispatch decisions with time coupling constraints (ramp limits and energy storage) and transport flow constraints to represent power transmission limits between multiple geospatial regions.

Note that detailed capacity expansion planning models can easily become large-scale constrainted optimization problems that push the limits of computational tractability. As such, these models commonly employ a range of abstraction methods along several dimensions, including temporal resolution, operational detail, and network/geospatial detail.

<img src="img/dimensionality.png" style="width: 450px; height: auto" align="left">

*Image source: [Jenkins & Sepulveda (2017)](https://energy.mit.edu/publication/enhanced-decision-support-changing-electricity-landscape/)*

In this case, to keep this tutorial model relatively digestible and to keep the solution time rapid for use in an interactive notebook like this, the model presented here includes:

- Economic dispatch with chronological ramp & storage constraints;
- Linear network flow constraints between three aggregate model regions; and
- Representative time periods (10 sample days selected via a clustering adapted from Mallapragada et al. (2018), "[Impact of model resolution on scenario outcomes for electricity sector system expansion](https://doi.org/10.1016/j.energy.2018.08.015
)" *Energy* 163)

Additionally all capacity investment are continuous, rather than discrete, keeping this model a linear program (LP). 

In the `complex_expansion_data/` path, we provide several different sets of inputs using different sample time periods, for your use and experimentation, including 10 days (used as default below), 4 weeks, 8 weeks, 16 weeks, and 52 weeks (full 8760 hours). Alter the `inputs_path` parameter below to select a different time series if desired.

All input data for this model are from the open source [PowerGenome](https://github.com/gschivley/PowerGenome) data platform.

This 'core' model can be extended in several directions to produce a more complicated/realistic capacity expansion planning model, incorporating discrete generator and/or transmission expansion, unit commitment constraints for thermal generators, other resources (e.g. hydropower, flexible demand, and much more), DC optimal power flow constraints, and many other elements. 

For examples of fully-developed complex capacity expansion models, see:
- GenX, described in Jenkins & Sepulveda (2017), "[Enhanced Decision Support for a Changing Electricity Landscape: The GenX Configurable Electricity Resource Capacity Expansion Model](https://energy.mit.edu/publication/enhanced-decision-support-changing-electricity-landscape/)," MIT Energy Initiative Working Paper 2017-10
- PyPSA, described in Brown et al. (2018), "[PyPSA: Python for Power System Analysis](https://doi.org/10.5334/jors.188)," *Journal of Open Research Software* and available open source at https://pypsa.org/
- SWITCH 2.0, described in Johnston et al. (2019), "[Switch 2.0: A modern platform for planning high-renewable power systems](https://doi.org/10.1016/j.softx.2019.100251)," *SoftwareX* 10 and available open source at http://switch-model.org/

We will dispense with a written mathematical formulation for this model, and proceed directly to the implementation in JuMP below.

## LOAD PACKAGES

In [2]:
# Uncomment this line if you need to install these packages:
# import Pkg; Pkg.add("Insert Package Name here");
using JuMP, HiGHS
using Plots; plotly()
using DataFrames, CSV

## LOAD INPUTS

In [3]:
# Read input data for a case with 10 sample days of data
inputs_path = "complex_expansion_data/10_days/"
  # Generators (and storage) data:
generators = DataFrame(CSV.File(joinpath(inputs_path, "Generators_data.csv")))
  # Many of the columns in the input data will be unused (this is input format for the GenX model)
  # Select the ones we want for this model
generators = select(generators, :R_ID, :Resource, :zone, :THERM, :DISP, :NDISP, :STOR, :HYDRO, :RPS, :CES,
                    :Commit, :Existing_Cap_MW, :Existing_Cap_MWh, :Cap_size, :New_Build, :Max_Cap_MW,
                    :Inv_cost_per_MWyr, :Fixed_OM_cost_per_MWyr, :Inv_cost_per_MWhyr, :Fixed_OM_cost_per_MWhyr,
                    :Var_OM_cost_per_MWh, :Start_cost_per_MW, :Start_fuel_MMBTU_per_MW, :Heat_rate_MMBTU_per_MWh, :Fuel,
                    :Min_power, :Ramp_Up_percentage, :Ramp_Dn_percentage, :Up_time, :Down_time,
                    :Eff_up, :Eff_down);
  # Set of all generators
G = generators.R_ID;
# Uncomment this line to explore the data if you wish:
# show(generators, allrows=true, allcols=true)

In [4]:
  # Read demand input data and record parameters
demand_inputs = DataFrame(CSV.File(joinpath(inputs_path, "Load_data.csv")))
# Value of lost load (cost of involuntary non-served energy)
VOLL = demand_inputs.Voll[1]
  # Set of price responsive demand (non-served energy) segments
S = convert(Array{Int64}, collect(skipmissing(demand_inputs.Demand_segment))) 
#NOTE:  collect(skipmising(input)) is needed here in several spots because the demand inputs are not 'square' (different column lengths)

  # Data frame for price responsive demand segments (nse)
  # NSE_Cost = opportunity cost per MWh of demand curtailment
  # NSE_Max = maximum % of demand that can be curtailed in each hour
  # Note that nse segment 1 = involuntary non-served energy (load shedding) at $9000/MWh
  # and segment 2 = one segment of voluntary price responsive demand at $600/MWh (up to 7.5% of demand)
nse = DataFrame(Segment=S, 
                NSE_Cost = VOLL.*collect(skipmissing(demand_inputs.Cost_of_demand_curtailment_perMW)),
                NSE_Max = collect(skipmissing(demand_inputs.Max_demand_curtailment)))

  # Set of sequential hours per sub-period
hours_per_period = convert(Int64, demand_inputs.Hours_per_period[1])
  # Set of time sample sub-periods (e.g. sample days or weeks)
P = convert(Array{Int64}, 1:demand_inputs.Subperiods[1])
  # Sub period cluster weights = number of hours represented by each sample period
W = convert(Array{Int64}, collect(skipmissing(demand_inputs.Sub_Weights)))
  # Set of all time steps
T = convert(Array{Int64}, demand_inputs.Time_index)
  # Create vector of sample weights, representing how many hours in the year
  # each hour in each sample period represents
sample_weight = zeros(Float64, size(T,1))
t=1
for p in P
    for h in 1:hours_per_period
        sample_weight[t] = W[p]/hours_per_period
        t=t+1
    end
end

  # Set of zones 
Z = convert(Array{Int64}, 1:3)

  # Load/demand time series by zone (TxZ array)
demand = select(demand_inputs, :Load_MW_z1, :Load_MW_z2, :Load_MW_z3);
# Uncomment this line to explore the data if you wish:
# show(demand, allrows=true, allcols=true)

In [5]:
  # Read generator capacity factors by hour (used for variable renewables)
  # There is one column here for each resource (row) in the generators DataFrame
variability = DataFrame(CSV.File(joinpath(inputs_path, "Generators_variability.csv")))
  # Drop the first column with row indexes, as these are unecessary
variability = variability[:,2:ncol(variability)];
# Uncomment this line to explore the data if you wish:
# show(variability, allrows=true, allcols=true)

In [6]:
  # Read fuels data
fuels = DataFrame(CSV.File(joinpath(inputs_path, "Fuels_data.csv")));
# Uncomment this line to explore the data if you wish:
# show(fuels, allrows=true, allcols=true);

In [7]:
  # Read network data
network = DataFrame(CSV.File(joinpath(inputs_path, "Network.csv")));
  #Again, there is a lot of entries in here we will not use (formatted for GenX inputs), so let's select what we want
  # Array of network zones (z1, z2, z3)
zones = collect(skipmissing(network.Network_zones))
  # Network map showing lines connecting zones
lines = select(network[1:2,:], 
    :Network_lines, :z1, :z2, :z3, 
    :Line_Max_Flow_MW, :Line_Min_Flow_MW, :Line_Loss_Percentage, 
    :Line_Max_Reinforcement_MW, :Line_Reinforcement_Cost_per_MW_yr)
  # Add fixed O&M costs for lines = 1/20 of reinforcement cost
lines.Line_Fixed_Cost_per_MW_yr = lines.Line_Reinforcement_Cost_per_MW_yr./20
  # Set of all lines
L = convert(Array{Int64}, lines.Network_lines);
# Uncomment this line to explore the data if you wish:
# show(lines, allrows=true, allcols=true)

In [8]:
# Calculate generator (and storage) total variable costs, start-up costs, 
# and associated CO2 per MWh and per start
generators.Var_Cost = zeros(Float64, size(G,1))
generators.CO2_Rate = zeros(Float64, size(G,1))
generators.Start_Cost = zeros(Float64, size(G,1))
generators.CO2_Per_Start = zeros(Float64, size(G,1))
for g in G
    # Variable cost ($/MWh) = variable O&M ($/MWh) + fuel cost ($/MMBtu) * heat rate (MMBtu/MWh)
    generators.Var_Cost[g] = generators.Var_OM_cost_per_MWh[g] +
        fuels[fuels.Fuel.==generators.Fuel[g],:Cost_per_MMBtu][1]*generators.Heat_rate_MMBTU_per_MWh[g]
    # CO2 emissions rate (tCO2/MWh) = fuel CO2 content (tCO2/MMBtu) * heat rate (MMBtu/MWh)
    generators.CO2_Rate[g] = fuels[fuels.Fuel.==generators.Fuel[g],:CO2_content_tons_per_MMBtu][1]*generators.Heat_rate_MMBTU_per_MWh[g]
    # Start-up cost ($/start/MW) = start up O&M cost ($/start/MW) + fuel cost ($/MMBtu) * start up fuel use (MMBtu/start/MW) 
    generators.Start_Cost[g] = generators.Start_cost_per_MW[g] +
        fuels[fuels.Fuel.==generators.Fuel[g],:Cost_per_MMBtu][1]*generators.Start_fuel_MMBTU_per_MW[g]
    # Start-up CO2 emissions (tCO2/start/MW) = fuel CO2 content (tCO2/MMBtu) * start up fuel use (MMBtu/start/MW) 
    generators.CO2_Per_Start[g] = fuels[fuels.Fuel.==generators.Fuel[g],:CO2_content_tons_per_MMBtu][1]*generators.Start_fuel_MMBTU_per_MW[g]
end
# Note: after this, we don't need the fuels Data Frame again...

In [9]:
# Drop hydropower and biomass plants from generators set for simplicity 
# (these are a small share of total capacity, ~500 MW
G = intersect(generators.R_ID[.!(generators.HYDRO.==1)],G)
G = intersect(generators.R_ID[.!(generators.NDISP.==1)],G);

## SETS

In [10]:
# Organize all of our data in one place...
# This code block is unecessary, but after all of the input steps above
# writing it all here helps us see all of the sets and parameter DataFrames.

# SUBSETS
  # By naming convention, all sets are single capital letters
G # Set of all generators
S # Set of all non-served energy (price responsive demand) segments
P  # Set of time sample sub-periods (e.g. sample days or weeks)
W  # Sub period cluster weights = number of periods (days/weeks) represented by each sample period
T  # Set of all time steps 
Z  # Set of zones (by number)
L;  # Set of all lines

## PARAMETERS

In [11]:
# PARAMETER DataFrames
  # By naming convention, all parameter data frames are lowercase
nse        # non-served energy parameters (by s in S)
generators # generation (and storage) parameters (by g in G)
demand     # demand parameters (by t in T)
zones      # network zones (by z in Z)
lines;      # transmission lines (by l in L)

## SUBSETS

In [12]:
#SUBSETS
  # By naming convention, all subsets are UPPERCASE

  # Subset of G of all thermal resources subject to unit commitment constraints
UC = intersect(generators.R_ID[generators.Commit.==1], G)
  # Subset of G NOT subject to unit commitment constraints
ED = intersect(generators.R_ID[.!(generators.Commit.==1)], G)
  # Subset of G of all storage resources
STOR = intersect(generators.R_ID[generators.STOR.>=1], G)
  # Subset of G of all variable renewable resources
VRE = intersect(generators.R_ID[generators.DISP.==1], G)
  # Subset of all new build resources
NEW = intersect(generators.R_ID[generators.New_Build.==1], G)
  # Subset of all existing resources
OLD = intersect(generators.R_ID[.!(generators.New_Build.==1)], G)
  # Subset of all RPS qualifying resources
RPS = intersect(generators.R_ID[generators.RPS.==1], G);

# Notes: findall(x->x in A, B) returns the intersection of two vectors A and B

## DEFINE MODEL

In [13]:
# LP model using HiGHS solver
Expansion_Model =  Model(HiGHS.Optimizer);

## DECISION VARIABLES

In [14]:
# DECISION VARIABLES
  # By naming convention, all decision variables start with v and then are in UPPER_SNAKE_CASE

# Capacity decision variables
@variables(Expansion_Model, begin
        vCAP[g in G]            >= 0     # power capacity (MW)
        vRET_CAP[g in OLD]      >= 0     # retirement of power capacity (MW)
        vNEW_CAP[g in NEW]      >= 0     # new build power capacity (MW)
        
        vE_CAP[g in STOR]       >= 0     # storage energy capacity (MWh)
        vRET_E_CAP[g in intersect(STOR, OLD)]   >= 0     # retirement of storage energy capacity (MWh)
        vNEW_E_CAP[g in intersect(STOR, NEW)]   >= 0     # new build storage energy capacity (MWh)
        
        vT_CAP[l in L]          >= 0     # transmission capacity (MW)
        vRET_T_CAP[l in L]      >= 0     # retirement of transmission capacity (MW)
        vNEW_T_CAP[l in L]      >= 0     # new build transmission capacity (MW)
end)

# Set upper bounds on capacity for renewable resources 
# (which are limited in each resource 'cluster')
for g in NEW[generators[NEW,:Max_Cap_MW].>0]
    set_upper_bound(vNEW_CAP[g], generators.Max_Cap_MW[g])
end

# Set upper bounds on transmission capacity expansion
for l in L
    set_upper_bound(vNEW_T_CAP[l], lines.Line_Max_Reinforcement_MW[l])
end

# Operational decision variables
@variables(Expansion_Model, begin
        vGEN[T,G]       >= 0  # Power generation (MW)
        vCHARGE[T,STOR] >= 0  # Power charging (MW)
        vSOC[T,STOR]    >= 0  # Energy storage state of charge (MWh)
        vNSE[T,S,Z]     >= 0  # Non-served energy/demand curtailment (MW)
        vFLOW[T,L]      # Transmission line flow (MW); 
          # note line flow is positive if flowing
          # from source node (indicated by 1 in zone column for that line) 
          # to sink node (indicated by -1 in zone column for that line); 
          # flow is negative if flowing from sink to source.
end)

(2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  231, 232, 233, 234, 235, 236, 237, 238, 239, 240]
    Dimension 2, [1, 4, 5, 6, 7, 8, 9, 11, 12, 13  …  46, 47, 48, 49, 50, 51, 52, 53, 54, 55]
And data, a 240×52 Matrix{VariableRef}:
 vGEN[1,1]    vGEN[1,4]    vGEN[1,5]    …  vGEN[1,54]    vGEN[1,55]
 vGEN[2,1]    vGEN[2,4]    vGEN[2,5]       vGEN[2,54]    vGEN[2,55]
 vGEN[3,1]    vGEN[3,4]    vGEN[3,5]       vGEN[3,54]    vGEN[3,55]
 vGEN[4,1]    vGEN[4,4]    vGEN[4,5]       vGEN[4,54]    vGEN[4,55]
 vGEN[5,1]    vGEN[5,4]    vGEN[5,5]       vGEN[5,54]    vGEN[5,55]
 vGEN[6,1]    vGEN[6,4]    vGEN[6,5]    …  vGEN[6,54]    vGEN[6,55]
 vGEN[7,1]    vGEN[7,4]    vGEN[7,5]       vGEN[7,54]    vGEN[7,55]
 vGEN[8,1]    vGEN[8,4]    vGEN[8,5]       vGEN[8,54]    vGEN[8,55]
 vGEN[9,1]    vGEN[9,4]    vGEN[9,5]       vGEN[9,54]    vGEN[9,55]
 vGEN[10,1]   vGEN[10,4]   vGEN[10,5]      vGEN[10,54]   vGEN[10,55]
 vGEN[11,1]   vGE

## CONSTRAINTS

In [15]:
# CONSTRAINTS
  # By naming convention, all constraints start with c and then are TitleCase

# (1) Supply-demand balance constraint for all time steps and zones
@constraint(Expansion_Model, cDemandBalance[t in T, z in Z], 
        sum(vGEN[t,g] for g in intersect(generators[generators.zone.==z,:R_ID],G)) +
        sum(vNSE[t,s,z] for s in S) - 
        sum(vCHARGE[t,g] for g in intersect(generators[generators.zone.==z,:R_ID],STOR)) -
        demand[t,z] - 
        sum(lines[l,Symbol(string("z",z))] * vFLOW[t,l] for l in L) == 0
);
# Notes: 
# 1. intersect(generators[generators.zone.==z,:R_ID],G) is the subset of all 
# generators/storage located at zone z in Z.
# 2. sum(lines[l,Symbol(string("z",z))].*FLOW[l,t], l in L) is the net sum of 
# all flows out of zone z (net exports) 
# 3. We use Symbol(string("z",z)) to convert the numerical reference to z in Z
# to a Symbol in set {:z1, :z2, :z3} as this is the reference to the columns
# in the lines data for zone z indicating which whether z is a source or sink
# for each line l in L.

In [16]:
# (2-6) Capacitated constraints:
@constraints(Expansion_Model, begin
# (2) Max power constraints for all time steps and all generators/storage
    cMaxPower[t in T, g in G], vGEN[t,g] <= variability[t,g]*vCAP[g]
# (3) Max charge constraints for all time steps and all storage resources
    cMaxCharge[t in T, g in STOR], vCHARGE[t,g] <= vCAP[g]
# (4) Max state of charge constraints for all time steps and all storage resources
    cMaxSOC[t in T, g in STOR], vSOC[t,g] <= vE_CAP[g]
# (5) Max non-served energy constraints for all time steps and all segments and all zones
    cMaxNSE[t in T, s in S, z in Z], vNSE[t,s,z] <= nse.NSE_Max[s]*demand[t,z]
# (6a) Max flow constraints for all time steps and all lines
    cMaxFlow[t in T, l in L], vFLOW[t,l] <= vT_CAP[l]
# (6b) Min flow constraints for all time steps and all lines
    cMinFlow[t in T, l in L], vFLOW[t,l] >= -vT_CAP[l]
end)

(2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  231, 232, 233, 234, 235, 236, 237, 238, 239, 240]
    Dimension 2, [1, 4, 5, 6, 7, 8, 9, 11, 12, 13  …  46, 47, 48, 49, 50, 51, 52, 53, 54, 55]
And data, a 240×52 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 cMaxPower[1,1] : -0.1844 vCAP[1] + vGEN[1,1] ≤ 0      …  cMaxPower[1,55] : -vCAP[55] + vGEN[1,55] ≤ 0
 cMaxPower[2,1] : -0.1263 vCAP[1] + vGEN[2,1] ≤ 0         cMaxPower[2,55] : -vCAP[55] + vGEN[2,55] ≤ 0
 cMaxPower[3,1] : -0.0817 vCAP[1] + vGEN[3,1] ≤ 0         cMaxPower[3,55] : -vCAP[55] + vGEN[3,55] ≤ 0
 cMaxPower[4,1] : -0.0494 vCAP[1] + vGEN[4,1] ≤ 0         cMaxPower[4,55] : -vCAP[55] + vGEN[4,55] ≤ 0
 cMaxP

In [17]:
# (7-9) Total capacity constraints:
@constraints(Expansion_Model, begin
# (7a) Total capacity for existing units
    cCapOld[g in OLD], vCAP[g] == generators.Existing_Cap_MW[g] - vRET_CAP[g]
# (7b) Total capacity for new units
    cCapNew[g in NEW], vCAP[g] == vNEW_CAP[g]
        
# (8a) Total energy storage capacity for existing units
    cCapEnergyOld[g in intersect(STOR, OLD)], 
        vE_CAP[g] == generators.Existing_Cap_MWh[g] - vRET_E_CAP[g]
# (8b) Total energy storage capacity for existing units
    cCapEnergyNew[g in intersect(STOR, NEW)], 
        vE_CAP[g] == vNEW_E_CAP[g]
        
# (9) Total transmission capacity
    cTransCap[l in L], vT_CAP[l] == lines.Line_Max_Flow_MW[l] - vRET_T_CAP[l] + vNEW_T_CAP[l]
end)

(1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, [1, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16]
And data, a 13-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 cCapOld[1] : vCAP[1] + vRET_CAP[1] = 8495.9
 cCapOld[4] : vCAP[4] + vRET_CAP[4] = 13567.68
 cCapOld[5] : vCAP[5] + vRET_CAP[5] = 36734.21
 cCapOld[6] : vCAP[6] + vRET_CAP[6] = 4704.48
 cCapOld[7] : vCAP[7] + vRET_CAP[7] = 1118.5
 cCapOld[8] : vCAP[8] + vRET_CAP[8] = 5020
 cCapOld[9] : vCAP[9] + vRET_CAP[9] = 6979
 cCapOld[11] : vCAP[11] + vRET_CAP[11] = 454.5
 cCapOld[12] : vCAP[12] + vRET_CAP[12] = 650
 cCapOld[13] : vCAP[13] + vRET_CAP[13] = 3427.34
 cCapOld[14] : vCAP[14] + vRET_CAP[14] = 1207.96
 cCapOld[15] : vCAP[15] + vRET_CAP[15] = 94

In [18]:
# Because we are using time domain reduction via sample periods (days or weeks),
# we must be careful with time coupling constraints at the start and end of each
# sample period. 

 # First we record a subset of time steps that begin a sub period 
 # (these will be subject to 'wrapping' constraints that link the start/end of each period)
STARTS = 1:hours_per_period:maximum(T)        
 # Then we record all time periods that do not begin a sub period 
# (these will be subject to normal time couping constraints, looking back one period)
INTERIORS = setdiff(T,STARTS)

# (10-12) Time coupling constraints
@constraints(Expansion_Model, begin
    # (10a) Ramp up constraints, normal
    cRampUp[t in INTERIORS, g in G], 
        vGEN[t,g] - vGEN[t-1,g] <= generators.Ramp_Up_percentage[g]*vCAP[g]
    # (10b) Ramp up constraints, sub-period wrapping
    cRampUpWrap[t in STARTS, g in G], 
        vGEN[t,g] - vGEN[t+hours_per_period-1,g] <= generators.Ramp_Up_percentage[g]*vCAP[g]    
    
    # (11a) Ramp down, normal
    cRampDown[t in INTERIORS, g in G], 
        vGEN[t-1,g] - vGEN[t,g] <= generators.Ramp_Dn_percentage[g]*vCAP[g] 
    # (11b) Ramp down, sub-period wrapping
    cRampDownWrap[t in STARTS, g in G], 
        vGEN[t+hours_per_period-1,g] - vGEN[t,g] <= generators.Ramp_Dn_percentage[g]*vCAP[g]     
   
    # (12a) Storage state of charge, normal
    cSOC[t in INTERIORS, g in STOR], 
        vSOC[t,g] == vSOC[t-1,g] + generators.Eff_up[g]*vCHARGE[t,g] - vGEN[t,g]/generators.Eff_down[g]
    # (12a) Storage state of charge, wrapping
    cSOCWrap[t in STARTS, g in STOR], 
        vSOC[t,g] == vSOC[t+hours_per_period-1,g] + generators.Eff_up[g]*vCHARGE[t,g] - vGEN[t,g]/generators.Eff_down[g]
end)

(2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, [2, 3, 4, 5, 6, 7, 8, 9, 10, 11  …  231, 232, 233, 234, 235, 236, 237, 238, 239, 240]
    Dimension 2, [1, 4, 5, 6, 7, 8, 9, 11, 12, 13  …  46, 47, 48, 49, 50, 51, 52, 53, 54, 55]
And data, a 230×52 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 cRampUp[2,1] : -vCAP[1] - vGEN[1,1] + vGEN[2,1] ≤ 0        …  cRampUp[2,55] : -0.64 vCAP[55] - vGEN[1,55] + vGEN[2,55] ≤ 0
 cRampUp[3,1] : -vCAP[1] - vGEN[2,1] + vGEN[3,1] ≤ 0           cRampUp[3,55] : -0.64 vCAP[55] - vGEN[2,55] + vGEN[3,55] ≤ 0
 cRampUp[4,1] : -vCAP[1] - vGEN[3,1] + vGEN[4,1] ≤ 0           cRampUp[4,55] : -0.64 vCAP[55] - vGEN[3,55] + vGEN[4,55] ≤ 0
 cRampUp[5,1] : -vCAP[1] - vGEN[4,1] + vGEN[5

## OBJECTIVE FUNCTION

In [19]:
# The objective function is to minimize the sum of fixed costs associated with
# capacity decisions and variable costs associated with operational decisions

# Create expressions for each sub-component of the total cost (for later retrieval)
@expression(Expansion_Model, eFixedCostsGeneration,
     # Fixed costs for total capacity 
    sum(generators.Fixed_OM_cost_per_MWyr[g]*vCAP[g] for g in G) +
     # Investment cost for new capacity
    sum(generators.Inv_cost_per_MWyr[g]*vNEW_CAP[g] for g in NEW)
)
@expression(Expansion_Model, eFixedCostsStorage,
     # Fixed costs for total storage energy capacity 
    sum(generators.Fixed_OM_cost_per_MWhyr[g]*vE_CAP[g] for g in STOR) + 
     # Investment costs for new storage energy capacity
    sum(generators.Inv_cost_per_MWhyr[g]*vNEW_E_CAP[g] for g in intersect(STOR, NEW))
)
@expression(Expansion_Model, eFixedCostsTransmission,
     # Investment and fixed O&M costs for transmission lines
    sum(lines.Line_Fixed_Cost_per_MW_yr[l]*vT_CAP[l] +
        lines.Line_Reinforcement_Cost_per_MW_yr[l]*vNEW_T_CAP[l] for l in L)
)
@expression(Expansion_Model, eVariableCosts,
     # Variable costs for generation, weighted by hourly sample weight
    sum(sample_weight[t]*generators.Var_Cost[g]*vGEN[t,g] for t in T, g in G)
)
@expression(Expansion_Model, eNSECosts,
     # Non-served energy costs
    sum(sample_weight[t]*nse.NSE_Cost[s]*vNSE[t,s,z] for t in T, s in S, z in Z)
)
  
@objective(Expansion_Model, Min,
    eFixedCostsGeneration + eFixedCostsStorage + eFixedCostsTransmission +
    eVariableCosts + eNSECosts
);

## RUN MODEL

In [20]:
@time optimize!(Expansion_Model)

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
37442 rows, 14141 cols, 110872 nonzeros
37438 rows, 14141 cols, 110864 nonzeros
Presolve : Reductions: rows 37438(-5339); columns 14141(-1815); elements 110864(-12532)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 482(3.17783e+06); Du: 0(1.92253e-10) 0s
      13904     1.4196651126e+10 Pr: 0(0); Du: 0(2.29937e-11) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 13904
Objective value     :  1.4196651126e+10
HiGHS run time      :          0.87
  3.467034 seconds (3.19 M allocations: 192.977 MiB, 2.24% gc time, 70.46% compilation time: 4% of which was recompilation)


## EXTRACT RESULTS

In [21]:
# Record generation capacity and energy results
generation = zeros(size(G,1))
for i in 1:size(G,1)
    # Note that total annual generation is sumproduct of sample period weights and hourly sample period generation 
    generation[i] = sum(sample_weight.*value.(vGEN)[:,G[i]].data) 
end

# Total annual demand is sumproduct of sample period weights and hourly sample period demands
total_demand = sum(sum.(eachcol(sample_weight.*demand)))
# Maximum aggregate demand is the maximum of the sum of total concurrent demand in each hour
peak_demand = maximum(sum(eachcol(demand)))
MWh_share = generation./total_demand.*100
cap_share = value.(vCAP).data./peak_demand.*100
generator_results = DataFrame(
    ID = G, 
    Resource = generators.Resource[G],
    Zone = generators.zone[G],
    Total_MW = value.(vCAP).data,
    Start_MW = generators.Existing_Cap_MW[G],
    Change_in_MW = value.(vCAP).data.-generators.Existing_Cap_MW[G],
    Percent_MW = cap_share,
    GWh = generation/1000,
    Percent_GWh = MWh_share
)

# Record energy storage energy capacity results (MWh)
storage_results = DataFrame(
    ID = STOR, 
    Zone = generators.zone[STOR],
    Resource = generators.Resource[STOR],
    Total_Storage_MWh = value.(vE_CAP).data,
    Start_Storage_MWh = generators.Existing_Cap_MWh[STOR],
    Change_in_Storage_MWh = value.(vE_CAP).data.-generators.Existing_Cap_MWh[STOR],
)


# Record transmission capacity results
transmission_results = DataFrame(
    Line = L, 
    Total_Transfer_Capacity = value.(vT_CAP).data,
    Start_Transfer_Capacity = lines.Line_Max_Flow_MW,
    Change_in_Transfer_Capacity = value.(vT_CAP).data.-lines.Line_Max_Flow_MW,
)


## Record non-served energy results by segment and zone
num_segments = maximum(S)
num_zones = maximum(Z)
nse_results = DataFrame(
    Segment = zeros(num_segments*num_zones),
    Zone = zeros(num_segments*num_zones),
    NSE_Price = zeros(num_segments*num_zones),
    Max_NSE_MW = zeros(num_segments*num_zones),
    Total_NSE_MWh = zeros(num_segments*num_zones),
    NSE_Percent_of_Demand = zeros(num_segments*num_zones)
)
i=1
for s in S
    for z in Z
        nse_results.Segment[i]=s
        nse_results.Zone[i]=z
        nse_results.NSE_Price[i]=nse.NSE_Cost[s]
        nse_results.Max_NSE_MW[i]=maximum(value.(vNSE)[:,s,z].data)
        nse_results.Total_NSE_MWh[i]=sum(sample_weight.*value.(vNSE)[:,s,z].data)
        nse_results.NSE_Percent_of_Demand[i]=sum(sample_weight.*value.(vNSE)[:,s,z].data)/total_demand*100
        i=i+1
    end
end

# Record costs by component (in million dollars)
 # Note: because each expression evaluates to a single value, 
 # value.(JuMPObject) returns a numerical value, not a DenseAxisArray;
 # We thus do not need to use the .data extension here to extract numeric values
cost_results = DataFrame(
    Total_Costs = objective_value(Expansion_Model)/10^6,
    Fixed_Costs_Generation = value.(eFixedCostsGeneration)/10^6,
    Fixed_Costs_Storage = value.(eFixedCostsStorage)/10^6,
    Fixed_Costs_Transmission = value.(eFixedCostsTransmission)/10^6,
    Variable_Costs = value.(eVariableCosts)/10^6,
    NSE_Costs = value.(eNSECosts)/10^6
);

## Write results to file

In [22]:
# Output path [set path to desired output directory here]
outpath = "results" #/YOUR/PATH/HERE

# If output directory does not exist, create it
if !(isdir(outpath))
    mkdir(outpath)
end

CSV.write(joinpath(outpath, "generator_results.csv"), generator_results)
CSV.write(joinpath(outpath, "storage_results.csv"), storage_results)
CSV.write(joinpath(outpath, "transmission_results.csv"), transmission_results)
CSV.write(joinpath(outpath, "nse_results.csv"), nse_results)
CSV.write(joinpath(outpath, "cost_results.csv"), cost_results);

# Question: cap and trade or clean electricity standard policy

As a bonus question, implement a CO$_2$ cap and trade program or clean electricity standard policy constraint in your model. 

Note that CO$_2$ emissions from generation and thermal unit start-ups are already calculated in the above Notebook model code and recorded in the parameters `generators.CO2_Rate[g]` and `generators.CO2_Per_Start[g]`. 

Eligibility for a clean electricity standard is also recorded in the `generators.CES[g]` parameter.

Take care in your implementation to consider the role of time weights when using a reduced time series as we are here (not all hours are created equal!).

Run the model with 10 days of data and under increasing stringency (e.g. from 20% to 40%, to 60%, to 80%, to 100% emissions reduction or renewable energy requirement), and record and discuss results of each case. 

What do you notice about solution time and results as you change the stringency of the policy? How much does the policy increase costs at each level of stringency, relative to no policy? How much does it reduce CO$_2$ emissions? (You will need to add a calculation of CO$_2$ emissions to the model outputs recorded)